In [ ]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid")
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator)
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import numpy as np
from connector import getConnection

### Mean  index rate (monthly basis)

In [ ]:
conn = getConnection()

sql = """
SELECT d.date ,AVG(s.index_rate) as avg_index_rate  FROM sale s, date d
WHERE s.sale_date_key = d.date_key 
GROUP BY d.date
HAVING AVG(s.index_rate) IS NOT NULL
ORDER BY d.date
"""

df = sqlio.read_sql_query(sql, conn)


conn.close()
df.head()
df.plot(kind='line',x='date',y='avg_index_rate',figsize=(15,10))



### distribution of loans on maritial status

In [ ]:
conn = getConnection()

sql = """
SELECT c.marital_status , SUM(s.loan_amount) as loan_amount  FROM sale s, customer c
WHERE s.customer_key = c.customer_key
GROUP BY c.marital_status
"""

df = sqlio.read_sql_query(sql, conn)

df = df.set_index('marital_status')
df = df.transpose()

df.plot.bar(stacked=True,figsize=(15,10), rot = 0)
conn.close()
df.head()



### Distribution of loans by age

In [ ]:
conn = getConnection()

sql = """
SELECT (2020 - c.birth_year) as age ,SUM( s.loan_amount)as loan_amount
FROM sale s, customer c
WHERE s.customer_key = c.customer_key
GROUP BY AGE
"""

df = sqlio.read_sql_query(sql, conn)
bins = pd.cut(df['age'], [10, 20, 30,40,50,60,70,80])
df = df.groupby(bins)['loan_amount'].agg(['sum'])
conn.close()
df.head()
df.plot.bar(figsize=(15,10), rot = 0)



### Distribution by job

In [ ]:
conn = getConnection()

sql = """
SELECT c.job ,SUM( s.loan_amount)as loan_amount
FROM sale s, customer c
WHERE s.customer_key = c.customer_key
GROUP BY c.job
ORDER BY loan_amount
"""

df = sqlio.read_sql_query(sql, conn)
#bins = pd.cut(df['age'], [0, 20, 40, 60,80])
#df = df.groupby(bins)['loan_amount'].agg(['sum'])
df = df.set_index('job')
conn.close()
df.head()
df.plot.barh(figsize=(15,10), rot = 0)

### number of customer with  loan

In [ ]:
conn = getConnection()

sql = """
select num_loans.n_loans, count(*)
from
(SELECT c.customer_key ,COUNT( s.loan_amount)as n_loans
FROM customer c left outer join sale s on  s.customer_key = c.customer_key
GROUP BY c.customer_key
ORDER BY n_loans desc) 
 num_loans 
GROUP BY num_loans.n_loans

"""

df = sqlio.read_sql_query(sql, conn)
df = df.set_index('n_loans')
conn.close()
df.head()
df.plot.bar(figsize=(15,10), rot = 0)

### Distribution of loans on currency

In [ ]:
conn = getConnection()

sql = """
SELECT p.currency ,SUM(s.loan_amount)    
FROM sale s, product p    
WHERE p.product_key = s.product_key   
GROUP BY  p.currency; 
"""

df = sqlio.read_sql_query(sql, conn)
conn.close()
df = df.set_index('currency')
df.head()

df.plot.bar(figsize=(15,10), rot = 0)


### Distribution loans in region

In [ ]:
conn = getConnection()

sql = """
select * from (
SELECT b.region ,SUM(s.loan_amount)    
FROM sale s, branch b, product p    
WHERE s.branch_key = b.branch_key AND p.product_key = s.product_key   
GROUP BY  b.region) aaaa
; 
"""

df = sqlio.read_sql_query(sql, conn)
conn.close()
df = df.set_index('region')
df.head()

df.plot.bar(figsize=(15,10), rot = 0)
#display(df)


### distribuition of loans by province

In [ ]:
conn = getConnection()

sql = """
select * from (
SELECT b.province ,SUM(s.loan_amount)    
FROM sale s, branch b, product p    
WHERE s.branch_key = b.branch_key AND p.product_key = s.product_key   
GROUP BY  b.province
) aaaa
; 
"""

df = sqlio.read_sql_query(sql, conn)
conn.close()
df = df.set_index('province')
df.head()

df.plot.bar(figsize=(15,10), rot = 0)
#display(df)


### distribuition of loans by province in 2018,2019

In [ ]:
conn = getConnection()

sql = """
WITH p_y_data AS (
SELECT b.province , d.year, SUM(s.loan_amount) as loan_amount
FROM sale s, branch b, date d  
WHERE s.branch_key = b.branch_key AND s.sale_date_key = d.date_key
GROUP BY  b.province, d.year) 
select a.province, a.loan_amount AS year_2018, b.loan_amount AS year_2019 from p_y_data a,   p_y_data b
 where a.year = 2018 AND b.year = 2019 AND
 a.province = b.province
; 
"""

df = sqlio.read_sql_query(sql, conn)
conn.close()
df = df.set_index('province')
df.head()

df.plot.bar(figsize=(15,10), rot = 0,colormap='Dark2')
#display(df)


### Cumilative Sale

In [ ]:
conn = getConnection()

sql = """
SELECT date, sum(sale.loan_amount) OVER (ORDER BY date) FROM sale, date
where sale.sale_date_key = date.date_key
;
"""

df = sqlio.read_sql_query(sql, conn)


conn.close()
df.head()
df.plot(kind='line',x='date',figsize=(15,10))

### Sale/employee ratio in provinces

In [ ]:
conn = getConnection()

sql = """
SELECT b.province , (SUM(s.loan_amount)/SUM(b.num_employee)) as loan_per_employee
FROM sale s, branch b, date d  
WHERE s.branch_key = b.branch_key AND s.sale_date_key = d.date_key
GROUP BY  b.province
; 
"""

df = sqlio.read_sql_query(sql, conn)
conn.close()
df = df.set_index('province')
df.head()

df.plot.bar(figsize=(15,10), rot = 0,colormap='Dark2')
#display(df)


### Interest per employee by province

In [ ]:
conn = getConnection()

sql = """
SELECT  b.province , SUM(i.interest)/SUM(b.num_employee) AS "interest income/employee"
FROM installment i, branch b
WHERE i.sale_branch_key = b.branch_key
GROUP BY  b.province
"""

df = sqlio.read_sql_query(sql, conn)
df = df.set_index('province')
df.head()
conn.close()
df.plot(kind="bar",figsize=(15,10), rot = 0, colormap='Dark2')


### Decalage

In [ ]:
conn = getConnection()

sql = """
select d.year,  sum(residual_capital) as "residual capital"
  from installment  i
  join "date" d on i.due_date_key = d.date_key
  group by d.year
  order by d.year
;
"""

df = sqlio.read_sql_query(sql, conn)
df['year'] = pd.to_datetime(df['year'], format='%Y')

df = df.set_index('year')

conn.close()
df.head()
df.plot(kind='line',figsize=(15,10))


### Total interest by year

In [ ]:
conn = getConnection()

sql = """
select d.year,  sum(interest) AS "interests" 
  from installment  i
  join "date" d on i.due_date_key = d.date_key
  group by d.year
  order by d.year
;
"""

df = sqlio.read_sql_query(sql, conn)
df['year'] = pd.to_datetime(df['year'], format='%Y')

df = df.set_index('year')

conn.close()
df.head()
df.plot(kind='line',figsize=(15,10), colormap='Dark2')
